In [1]:
import numpy as np
import pandas as pd



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import torch
import transformers


import warnings
import time
warnings.filterwarnings('ignore')


In [2]:
sst2_path = '../../data/SST-2'

pre_trained_weights_type = 'distilbert-base-uncased'

In [3]:
sst2_train = pd.read_csv(sst2_path + '/train.tsv', delimiter='\t')

In [4]:
sst2_train.shape

(67349, 2)

In [5]:
sst2_train.head(10)

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0
5,that 's far too tragic to merit such superfici...,0
6,demonstrates that the director of such hollywo...,1
7,of saucy,1
8,a depressed fifteen-year-old 's suicidal poetry,0
9,are more deeply thought through than in most `...,1


In [6]:
# Balance of Binary Classification Data 
sst2_train['label'].value_counts()

1    37569
0    29780
Name: label, dtype: int64

### Import Model Class , tokenizer Class and Pretrained Weights for Fine Tuning 

In [7]:
from transformers import DistilBertModel , DistilBertTokenizer

In [8]:
# Setting pre_trained weights 

pre_trained_weights = pre_trained_weights_type 

# Call the Model 
distilbert_model = DistilBertModel.from_pretrained(pre_trained_weights)

# Call the Tokenizer 

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(pre_trained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### https://huggingface.co/docs/transformers/training